In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
train_input = pd.read_csv("/content/X_train_6GWGSxz.csv")
train_output = pd.read_csv("/content/y_train_2G60rOL.csv")

test_input = pd.read_csv("/content/X_test_c2uBt2s.csv")


FileNotFoundError: [Errno 2] No such file or directory: '/content/X_train_6GWGSxz.csv'

train_input['time_step'] = pd.to_datetime(train_input['time_step'])
train_output['time_step'] = pd.to_datetime(train_output['time_step'])
test_input['time_step'] = pd.to_datetime(test_input['time_step'])


In [ ]:
train_input['time_step'] = pd.to_datetime(train_input['time_step'])
train_output['time_step'] = pd.to_datetime(train_output['time_step'])
test_input['time_step'] = pd.to_datetime(test_input['time_step'])


In [ ]:
print("Colonnes input : ", train_input.columns.tolist())
print("Colonnes output :", train_output.columns.tolist())


In [ ]:
train_input.head(120)



In [ ]:

plt.figure(figsize=(15, 4))
plt.plot(train_input['time_step'][:2000], train_input['consumption'][:2000])
plt.title("Consommation totale - premiers 2000 points")
plt.xlabel("Temps")
plt.ylabel("W")
plt.show()


In [ ]:
missing_input = train_input.isna().mean() * 100
missing_output = train_output.isna().mean() * 100

print("Taux de NA - Input (%):\n", missing_input)
print("\nTaux de NA - Output (%):\n", missing_output)


In [ ]:
meteo_cols = ['visibility','temperature','humidity','humidex','windchill','wind','pressure']

# Ensuite forward-fill pour remplir les minutes entre heures
train_input[meteo_cols] = train_input[meteo_cols].bfill()
# Ensuite forward-fill pour remplir les minutes entre heures
train_input[meteo_cols] = train_input[meteo_cols].ffill()



In [ ]:
train_output[['washing_machine','fridge_freezer','TV','kettle']] = \
    train_output[['washing_machine','fridge_freezer','TV','kettle']].ffill().bfill()
train_input['consumption'] = train_input['consumption'].ffill().bfill()




In [ ]:
df = train_input.merge(train_output, on='time_step')
df.head()
df.shape



In [ ]:

# ============================

# Extraire informations temporelles
df['weekday'] = df['time_step'].dt.weekday   # 0 = lundi
df['hour'] = df['time_step'].dt.hour
df['minute'] = df['time_step'].dt.minute

# Weekend (0/1)
df['is_weekend'] = (df['weekday'] >= 5).astype(int)

# Encodage circulaire de l'heure
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

# Dummies pour jour de la semaine
df = pd.get_dummies(df, columns=['weekday'], prefix='day')

# ============================
# 5. SÉPARATION X et Y
# ============================

# Sorties
Y = df[['washing_machine','fridge_freezer','TV','kettle']]

# Entrées
X = df[['consumption','visibility','temperature','humidity',
        'humidex','windchill','wind','pressure',
        'is_weekend','hour_sin','hour_cos']
      + [col for col in df.columns if col.startswith("day_")]]

print("X shape =", X.shape)
print("Y shape =", Y.shape)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Diviser en 80% pour l'entraînement et 20% pour la validation
X_train_split, X_val_split, Y_train_split, Y_val_split = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

# Liste des appareils
devices = ['washing_machine', 'fridge_freezer', 'TV', 'kettle']

# Dictionnaires pour stocker MAE et RMSE
mae_dict = {}
rmse_dict = {}

# Entraîner un modèle linéaire pour chaque appareil
for device in devices:
    model = LinearRegression()

    # Entraîner le modèle sur les données d'entraînement
    model.fit(X_train_split, Y_train_split[device])

    # Prédire les valeurs sur les données d'entraînement (ou de validation si tu veux évaluer pendant l'entraînement)
    Y_pred = model.predict(X_val_split)

    # Calculer le MAE (Mean Absolute Error)
    mae_dict[device] = mean_absolute_error(Y_val_split[device], Y_pred)

    # Calculer le RMSE (Root Mean Squared Error)
    rmse_dict[device] = np.sqrt(mean_squared_error(Y_val_split[device], Y_pred))

    # Afficher les résultats pour chaque appareil
    print(f"{device} MAE: {mae_dict[device]:.2f} W")
    print(f"{device} RMSE: {rmse_dict[device]:.2f} W")



Le modèle de régression linéaire pour la prédiction des consommations des appareils est performant pour certains appareils comme la machine à laver et la TV, avec un MAE inférieur au benchmark de 47.64 W. Cependant, des erreurs sont observées pour le frigo et la bouilloire, principalement en raison de leur comportement cyclique et impulsif. Ces limitations suggèrent que des modèles plus complexes, comme les réseaux neuronaux (LSTM, CNN), pourraient améliorer la précision pour ces appareils.

In [ ]:
benchmark = 47.64

for device in devices:
    print(f"{device} - Benchmark MAE: {benchmark:.2f} W")
    print(f"{device} - MAE: {mae_dict[device]:.2f} W (Better than Benchmark? {mae_dict[device] < benchmark})\n")


In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Normalisation des données
scaler = MinMaxScaler()

# Normaliser X_train et X_val entre 0 et 1 (sauf time_step)
X_train_scaled = scaler.fit_transform(X_train_split)
X_val_scaled = scaler.transform(X_val_split)


def create_lstm_dataset(X, Y, timesteps=50):
    X_lstm, Y_lstm = [], []

    for i in range(timesteps, len(X)):  # Commence à "timesteps" pour éviter les indices négatifs
        X_lstm.append(X[i-timesteps:i])  # Crée une séquence de `timesteps` observations
        Y_lstm.append(Y[i])  # La cible à prédire (la valeur à prédire à chaque `timesteps`)

    return np.array(X_lstm), np.array(Y_lstm)

# Créer les datasets pour LSTM
X_train_lstm, Y_train_lstm = create_lstm_dataset(X_train_scaled, Y_train_split)
X_val_lstm, Y_val_lstm = create_lstm_dataset(X_val_scaled, Y_val_split)

# Vérifier les formes des données
print(f"X_train_lstm shape: {X_train_lstm.shape}")
print(f"Y_train_lstm shape: {Y_train_lstm.shape}")
print(f"X_val_lstm shape: {X_val_lstm.shape}")
print(f"Y_val_lstm shape: {Y_val_lstm.shape}")


In [ ]:

print(f"Shape of Y: {Y_train_split.shape}")  # Taille des labels

# Affichons les premiers éléments de X et Y pour s'assurer que tout est OK

print("Premiers éléments de Y:", Y_train_split[:100])
